In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from flowtracks.io import iter_trajectories_ptvis
import glob
import pandas as pd
import seaborn as sns
import scipy.stats as st
import math
%matplotlib inline

In [ ]:
runs = [[0,1,2],[3,4,5]]
times4 = [[(177,267),(172,262),(182,262),(180,260),(190,250)],[],[],
          [(65,115),(55,105),(70,110),(45,95),(60,110)],
           [(50,110),(40,110),(35,105),(60,130),(40,110)],
           [(60,115),(0,75),(40,100),(55,120),(60,120)]]
times5 = [[(40,100),(50,110),(40,100),(40,100),(43,103)],
           [(145,215),(150,250),(0,120),(0,97),(30,115)],
           [(200,255),(155,244),(40,105),(175,235),(162,262)],
          [(50,100),(45,90),(40,95),(45,105),(30,80)],
           [(10,90),(30,75),(30,85),(25,95),(30,110)],
           [(35,90),(200,270),(15,110),(0,104),(0,104)]]
times6 = [[(1,91),(6,96),(10,100),(0,100),(0,100)],
           [(170,265),(170,255),(170,240),(165,250),(165,255)],
           [(5,85),(180,270),(172,267),(167,237),(180,260)],
          [(165,260),(0,80),(180,250),(180,250),(180,250)],
           [(180,255),(170,250),(170,240),(170,255),(160,245)],
           [(170,260),(175,240),(165,255),(0,104),(0,104)]]

V_4 = [0.0,0.0,0.0,6.26,5.51,5.19]
V_5 = [0.0,3.49,4.00,6.21,5.62,5.21] 
V_6 = [0.0,3.48,3.95,6.22,5.54,5.15]
wind_vel = [V_4,V_5,V_6] 

MP_4 = [6.2294,0,0,6.7134,7.0783,7.7943]
MP_5 = [7.7114,7.5392,7.7129,8.4741,8.2427,8.9891]
MP_6 = [8.9672,7.3680,8.4021,10.6217,11.7332,14.0009]
MP = [MP_4,MP_5,MP_6]

amplitude = [0.062, 0.075, 0.087]
numbers = np.zeros((3,6))

main_folder = "/mn/kadingir/reynar_000000/ptvs/"

In [ ]:
part = pd.DataFrame(columns=['time','r','x','y','z','u','v','w',
                             'acc_x','acc_y','wind','A','size','run'])
p = np.poly1d(np.array([-4.328125e-04,  2.568750e-02,  5.377800]))
resize = np.poly1d(np.array([0.05361976, 0.47352799, 0.69640069]))

for i in [0]:
    print(str(i))
    navn = main_folder+"A0%i_%i/res-all/ptv_is"
    for k in [5]:
        print(str(k))
        if k == 5:
            where = times5[0]
            ind = 1
            Umax = wind_vel[ind]
        elif k == 6:
            where = times6[0]
            ind = 2
            Umax = wind_vel[ind]
        elif k == 4:
            where = times4[0]
            ind = 0
            Umax = wind_vel[ind]
        fig,ax = plt.subplots(1,2,figsize=(8,4))
        for l in range(1,6):
            print('.',end="")
            inName = navn%(k,l)+".%d"
            name = inName[:-2]+"*"
            listOfFiles = glob.glob(name)
            N = len(listOfFiles)
            traj_min_len = 5 # in this particular example we have short trajectories
            start = where[l-1][0]
            end = where[l-1][1]
            #----cal traj.
            trajects = list(iter_trajectories_ptvis(inName,
                            first=10000+start, last=10000+end,
                                    traj_min_len=traj_min_len))
            C = 0
            count = 0
            for tr in trajects:
                if 3.*np.std(tr.pos()[:,0])> 6e-5 and 3.*np.std(tr.pos()[:,1])>6e-5 :
                    #Filter out trajectories that seem impossible
                    correlation_matrix = np.corrcoef(tr.pos()[:,0]**2, tr.pos()[:,1])
                    correlation_xy = correlation_matrix[0,1]
                    r_squared = correlation_xy**2
                    if r_squared >0.8: 
                        count+=1
                        T =tr.time()-10000-start
                        R = tr.pos()
                        t0,tn = tr.time()[0],tr.time()[-1]
                        Vel = tr.velocity()
                        Acc = tr.accel()
                        sizes = np.zeros(int(tn-t0)+1)
                        paso = 0
                        #Take one particle and follow through time
                        for m in range(int(t0),int(tn+1)):
                            fname = navn[:-6]%(k,l)+'rt_is.%d'%(m)
                            x,y,z = R[paso]
                            data = np.loadtxt(fname,skiprows=1,usecols=(1,2,3,4,5,6,7))
                            idx = np.where(abs(data[:,0]-x*1e3)<1e-3)
                            #Find ID of particle i ptv_is files
                            ID = np.array([0])
                            #Pixel to world scale
                            scale = p(z*1e3)
                            if len(idx[0])>1:
                                #print('More than one x')
                                piece = data[idx[0],:]
                                idy = np.where(abs(piece[:,1]-y*1e3)<1e-3)
                                if len(idy[0])>1:
                                    #print('More than one y')
                                    piece = data[idx[0][idy[0]]][:]
                                    idz = np.where(abs(piece[:,2]-z*1e3)<1e-3)
                                    if len(idz[0])>1:
                                        ID = idx[0][idy[0][idz[0]]]
                                    elif len(idz[0])==1:
                                        ID = idx[0][idy[0][idz[0]]]
                                elif len(idy[0])==1:
                                    ID = idx[0][idy[0]]
                            elif len(idx[0])<1:
                                print('No correspondence found. Perhaps epsilon value too small')
                            else:
                                ID = idx[0]
                            #Obtain pixel values in the 4 cameras from targets
                            ID1, ID2, ID3, ID4 = int(data[ID][0][3]),int(data[ID][0][4]),int(data[ID][0][5]),int(data[ID][0][6])
                            if ID1 < 0:
                                C1 = np.array([np.nan,np.nan])
                            else:
                                c1_name = main_folder+"/A0%i_%i/masked/cam1.%i_targets"%(k,l,m)
                                C1 = np.loadtxt(c1_name,skiprows=ID1+1,usecols=(4,5),max_rows=1)
                            if ID2 < 0:
                                C2 = np.array([np.nan,np.nan])
                            else:
                                c2_name = main_folder+"/A0%i_%i/masked/cam2.%i_targets"%(k,l,m)
                                C2 = np.loadtxt(c2_name,skiprows=ID2+1,usecols=(4,5),max_rows=1)
                            if ID3 < 0:
                                C3 = np.array([np.nan,np.nan])
                            else:
                                c3_name = main_folder+"/A0%i_%i/masked/cam3.%i_targets"%(k,l,m)
                                C3 = np.loadtxt(c3_name,skiprows=ID3+1,usecols=(4,5),max_rows=1)
                            if ID4 < 0:
                                C4 = np.array([np.nan,np.nan])
                            else:
                                c4_name = main_folder+"/A0%i_%i/masked/cam4.%i_targets"%(k,l,m)
                                C4 = np.loadtxt(c4_name,skiprows=ID4+1,usecols=(4,5),max_rows=1)
                            a = np.nanmean([C1[0],C2[0],C3[0],C4[0]])
                            b = np.nanmean([C1[1],C2[1],C3[1],C4[1]])
                            Rad = np.sqrt(a*b)
                            sizes[paso]=resize(Rad/scale)
                            paso+=1
                        for lon in range(len(T)): 
                            #Filter out velocities that are below resolution
                            if abs(Vel[lon,0]) > 2e-5 and abs(Vel[lon,1]) > 2e-5:
                                #Store data
                                part = part.append({'time':T[lon]/167,
                                             'x':R[lon,0]*10,'y':R[lon,1]*10+0.14,'z':R[lon,2]*10,
                                             'u':Vel[lon,0]*1670,'v':Vel[lon,1]*1670,'w':Vel[lon,2]*1670,
                                              'wind':Umax[0] ,'acc_x':Acc[lon,0]*278890,'acc_y':Acc[lon,1]*278890,
                                            'A':amplitude[ind] ,'size':sizes[lon],'run':l},
                                            ignore_index = True)
                                
                        if C<9:
                            C+=1
                        elif C==9:
                            C=0
            numbers[ind][0]=count